
## Diffusion Model

* So let's start by importing all required libraries

In [1]:
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import re
from collections import Counter

<hr style="border: 1px solid black;">

* Let's first read shakespeare data and see unique characters in it so that we can remove unwanted characters

In [2]:
with open("./data/shakespeare-sonnets.txt", "r") as f:
    print(set(f.read()))

{'3', 'l', '"', '?', 'V', 'u', 'y', 'I', 'k', 'U', '!', 'f', 'T', 't', 'S', 'J', 'W', 'v', '.', '7', '1', 'z', 'O', 'r', 'm', 'R', 'b', 'q', 'i', '9', 'E', 'w', ' ', '4', ',', '-', '\n', '6', 'g', 'H', 'j', 'a', '0', 'p', ';', 'M', ']', 'h', 'L', 'K', 'n', 'F', ':', 'B', 'c', 'Y', 'A', '8', 'P', '[', 's', 'C', '5', 'x', 'G', 'D', 'd', "'", '2', 'N', 'o', 'e'}


* As you can see from above that there are characters like square brackets and numbers which we dont want
* So let's start by first writing a function for cleaning text
* In this function we are going to remove 

In [3]:
def clean_shakespeare_text(text):
    """Function to clean given text"""
    # Removing numbers
    pattern = r'\d+'
    text = re.sub(pattern, "", text)
    # Removing square brackets
    text = re.sub(r'\[\]', '', text, flags=re.MULTILINE)
    # Splitting lines by next line character
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        # Removing spaces from start and end of line
        stripped_line = line.strip()
        # Skipping empty line
        if not stripped_line:
            continue
        cleaned_lines.append(stripped_line)
    return cleaned_lines

* Now let's test this function and see if it's working or not

In [4]:
clean_shakespeare_text("1\n abcdefg \n        1256           \nabc?")

['abcdefg', 'abc?']

* As you can see that the function is working as expected
* Now let's get our cleaned shakespeare text data by passing it through this function

In [5]:
cleaned_data = []
with open("./data/shakespeare-sonnets.txt", "r") as f:
    cleaned_data = clean_shakespeare_text(f.read())

* Now let's see first 10 lines of cleaned data and also see it's length

In [6]:
len(cleaned_data), cleaned_data[:10]

(2183,
 ['From fairest creatures we desire increase,',
  "That thereby beauty's rose might never die,",
  'But, as the riper should by time decease,',
  'His tender heir might bear his memory.',
  'But thou, contracted to thine own bright eyes,',
  "Feed'st thy light's flame with self-substantial fuel,",
  'Making a famine where abundance lies,',
  'Thyself thy foe, to thy sweet self too cruel.',
  "Thou that art now the world's fresh ornament",
  'And only herald to the gaudy spring'])

* Now let's write a function to prepare samples of shakespeare text
* This function will have four parameters which says whether to include single line, two lines (couplets), four lines (quatrain) and fourteen lines (sonnet) and return samples accordingly
* So we are using different samples because :
    - Single line - learn vocabulary
    - Two lines (couplet) - learn how two lines connect
    - Four lines (quatrain) - learn rhyme patterns in line
    - Fourteen lines (sonnet) - learn poem structure

In [7]:
def prepare_mixed_samples(lines, include_single=True, include_couplet=True, include_quatrain=True, include_sonnet=True):
    """Function to return requested length samples"""
    all_samples = []
    # If single is true then include single line in all_samples
    if include_single:
        for line in lines:
            all_samples.append({'text': line, 'type': 'single', 'num_lines': 1})
    # If couplet is true then include two lines in all_samples
    if include_couplet:
        for i in range(0, len(lines) - 1, 2):
            couplet = f"{lines[i]}\n{lines[i+1]}"
            all_samples.append({'text': couplet, 'type': 'couplet', 'num_lines': 2})
    # If quatrain is true then include four lines in all_samples
    if include_quatrain:
        for i in range(0, len(lines) - 3, 4):
            quatrain = '\n'.join(lines[i:i+4])
            all_samples.append({'text': quatrain, 'type': 'quatrain', 'num_lines': 4})
    # If sonnet is true then include fourteen lines in all_samples
    if include_sonnet:
        for i in range(0, len(lines) - 13, 14):
            sonnet = '\n'.join(lines[i:i+14])
            all_samples.append({'text': sonnet, 'type': 'sonnet', 'num_lines': 14})
    return all_samples

* Now let's test this function

In [8]:
test_data = ["abcd" for _ in range(20)]
prepare_mixed_samples(test_data, include_single=True, include_couplet=False, include_quatrain=False, include_sonnet=False)

[{'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines

In [9]:
prepare_mixed_samples(test_data, include_single=True, include_couplet=True, include_quatrain=False, include_sonnet=False)

[{'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines

In [10]:
prepare_mixed_samples(test_data, include_single=True, include_couplet=True, include_quatrain=True, include_sonnet=False)

[{'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines

In [11]:
prepare_mixed_samples(test_data, include_single=True, include_couplet=True, include_quatrain=True, include_sonnet=True)

[{'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines': 1},
 {'text': 'abcd', 'type': 'single', 'num_lines

* As you can see from above that the function is working as expected
* So let's pass cleaned shakespeare data to this function and get all samples

In [12]:
all_data_samples = prepare_mixed_samples(cleaned_data, include_single=True, include_couplet=True, include_quatrain=False, include_sonnet=False)
all_data_samples

[{'text': 'From fairest creatures we desire increase,',
  'type': 'single',
  'num_lines': 1},
 {'text': "That thereby beauty's rose might never die,",
  'type': 'single',
  'num_lines': 1},
 {'text': 'But, as the riper should by time decease,',
  'type': 'single',
  'num_lines': 1},
 {'text': 'His tender heir might bear his memory.',
  'type': 'single',
  'num_lines': 1},
 {'text': 'But thou, contracted to thine own bright eyes,',
  'type': 'single',
  'num_lines': 1},
 {'text': "Feed'st thy light's flame with self-substantial fuel,",
  'type': 'single',
  'num_lines': 1},
 {'text': 'Making a famine where abundance lies,',
  'type': 'single',
  'num_lines': 1},
 {'text': 'Thyself thy foe, to thy sweet self too cruel.',
  'type': 'single',
  'num_lines': 1},
 {'text': "Thou that art now the world's fresh ornament",
  'type': 'single',
  'num_lines': 1},
 {'text': 'And only herald to the gaudy spring',
  'type': 'single',
  'num_lines': 1},
 {'text': 'Within thine own bud buriest thy co

In [13]:
Counter(sample['type'] for sample in all_data_samples)

Counter({'single': 2183, 'couplet': 1091})

* Now let's create shakespeare dataset class which will take all samples and precompute and then return contextualized BERT embedding for text and also for first n POS (Part of speech) words
* Here for text embeddings (i.e. embeddings for samples) we are contextualizing from whole text (sample) and for POS embedding we are contextualizing from POS word only as at time of generation POS embeddings are going to be contextualized from POS words so to keep things similar for POS we are contextlaizing based on POS words only
* We also add normalization to embedding because diffusion model works best with normalized data
* Noise that we add to this data is sampled from gausian distribution having mean of 0 and standard deviation of 1 so normalizing embedding data leads to stable training and also match noise mean and standard deviation

In [14]:
class ShakespeareDiffusionDataset(Dataset):
    """Dataset with normalization for diffusion which returns BERT embeddings"""
    def __init__(self, samples, model_name='bert-base-uncased', max_length=64, n_pos_words=3, device='cuda'):
        self.samples = samples
        self.model_name = model_name
        self.max_length = max_length
        self.n_pos_words = n_pos_words
        self.device = device
        # Loading model and tokenizer
        print(f"\nLoading BERT model: {model_name}")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.model.eval()
        self.hidden_dim = self.model.config.hidden_size
        print(f"Hidden dimension: {self.hidden_dim}")
        # Precomputing all embeddings
        print(f"Pre-computing embeddings for {len(samples)} samples...")
        self.raw_embeddings, self.pos_embeddings = self._precompute_all()
        # Calculating normalization statistics from the data
        print("Calculating normalization statistics.")
        all_emb = torch.stack(self.raw_embeddings)
        self.emb_mean = all_emb.mean().item()
        self.emb_std = all_emb.std().item()
        print(f"Raw embedding stats: mean={self.emb_mean:.4f}, std={self.emb_std:.4f}")
        # Normalizing embeddings to mean=0, std=1
        self.embeddings = [(e - self.emb_mean) / self.emb_std for e in self.raw_embeddings]
        # Verifying normalization
        all_norm = torch.stack(self.embeddings)
        print(f"  Normalized stats: mean={all_norm.mean():.4f}, std={all_norm.std():.4f}")
        print(f"Done! Dataset ready.\n")
        # Freeing GPU memory
        del self.model
        torch.cuda.empty_cache()
    
    def _get_text_embedding(self, text):
        """Function to get BERT contextual embeddings for text"""
        encoded = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_length).to(self.device)
        outputs = self.model(**encoded)
        embedding = outputs.last_hidden_state.squeeze(0).cpu()
        return embedding
    
    def _get_pos_embedding(self, text):
        """Function to get embeddings for POS words only"""
        clean_text = text.replace('\n', ' ')
        words = clean_text.split()[:self.n_pos_words]
        while len(words) < self.n_pos_words:
            words.append(self.tokenizer.pad_token if self.tokenizer.pad_token else '[PAD]')
        pos_text = ' '.join(words)
        encoded = self.tokenizer(pos_text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.n_pos_words + 2).to(self.device)
        outputs = self.model(**encoded)
        pos_emb = outputs.last_hidden_state.squeeze(0)[1:self.n_pos_words + 1, :]
        return pos_emb.cpu()
    
    def _precompute_all(self):
        """Function to precompute embeddings for all samples"""
        all_embeddings = []
        all_pos_embeddings = []
        with torch.no_grad():
            for sample in tqdm(self.samples):
                text = sample['text']
                text_emb = self._get_text_embedding(text)
                all_embeddings.append(text_emb)
                pos_emb = self._get_pos_embedding(text)
                all_pos_embeddings.append(pos_emb)
        return all_embeddings, all_pos_embeddings
    
    def get_normalization_stats(self):
        """Function which returns normalization statistics for use during generation"""
        return self.emb_mean, self.emb_std
    
    def __len__(self):
        """Function to return length of samples in dataset"""
        return len(self.embeddings)
    
    def __getitem__(self, idx):
        """Function to return requested items embeddings"""
        return {'embeddings': self.embeddings[idx], 'pos_embeddings': self.pos_embeddings[idx]}

* Now that we have written dataset class let's make it's instance and provide it to dataloader which will divide given dataset into batches and shuffle it as well

In [15]:
model_name = 'bert-base-uncased'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_pos_words = 3
max_length = 64
batch_size = 32

dataset = ShakespeareDiffusionDataset(samples=all_data_samples, model_name=model_name, max_length=max_length, n_pos_words=n_pos_words, device=device)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)


Loading BERT model: bert-base-uncased
Hidden dimension: 768
Pre-computing embeddings for 3274 samples...


100%|██████████| 3274/3274 [00:24<00:00, 131.01it/s]


Calculating normalization statistics.
Raw embedding stats: mean=-0.0089, std=0.4152
  Normalized stats: mean=-0.0000, std=1.0000
Done! Dataset ready.



* Let's view the first sample embedding returned by dataset 

In [16]:
dataset[0]

{'embeddings': tensor([[ 0.2457,  0.7117,  0.1252,  ..., -0.3991, -0.1186,  0.8122],
         [ 0.6243,  1.4963,  1.0853,  ..., -0.2058,  1.3613,  1.0909],
         [ 1.5481, -0.8929,  2.1297,  ..., -1.5306, -0.7201, -1.4385],
         ...,
         [ 0.5821, -0.1412,  0.7124,  ..., -0.1153,  0.6964, -0.6126],
         [ 0.3272, -0.1976,  0.7915,  ..., -0.0949,  0.6067, -0.5615],
         [-0.9759, -1.0914, -0.3940,  ...,  0.3828,  0.5988, -0.0785]]),
 'pos_embeddings': tensor([[ 0.1607, -0.6030,  0.1060,  ..., -0.1467,  0.6751,  0.4106],
         [ 0.2393, -0.7291,  1.0812,  ..., -0.6851,  0.1290, -0.3506],
         [-0.7258, -0.9580, -0.3143,  ...,  0.1221,  1.0462, -0.0555]])}

* Now that we have our dataset ready let's create our Unet model
* Before writing unet let's write utility networks required by Unet
* So let's first write class for performing timestep embedding in Unet (Using professor's code)

In [17]:
class SinusoidalPositionEmbeddings(nn.Module):
    """Time step embeddings for diffusion"""
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings

* Now let's write dilated convolution block (Using professor's code)

In [18]:
class DilatedConvBlock(nn.Module):
    """Dilated convolution block with POS cross-attention conditioning"""
    def __init__(self, channels, dilation, time_emb_dim, pos_dim, num_heads=4, dropout=0.1):
        super().__init__()
        
        # Dilated convolution for local text patterns
        self.conv1 = nn.Conv1d(
            channels, channels,
            kernel_size=3,
            dilation=dilation,
            padding=dilation
        )
        self.conv2 = nn.Conv1d(
            channels, channels,
            kernel_size=3,
            dilation=dilation,
            padding=dilation
        )
        
        # Time embedding projection
        self.time_mlp = nn.Sequential(
            nn.Linear(time_emb_dim, channels),
            nn.SiLU(),
            nn.Linear(channels, channels)
        )

        # Project POS embedding to match this layer's channel dimension
        self.pos_proj = nn.Linear(pos_dim, channels)
        
        # Cross-attention to POS embeddings
        # print("MHA channels:", channels)
        # print("MHA num_heads:", num_heads)
        self.pos_cross_attn = nn.MultiheadAttention(
            channels,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        
        # Normalization layers
        self.norm1 = nn.GroupNorm(8, channels)
        self.norm2 = nn.GroupNorm(8, channels)
        self.norm3 = nn.LayerNorm(channels)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Activation
        self.act = nn.SiLU()
        
    def forward(self, x, time_emb, pos_embedding):
        """
        x: [batch, channels, seq_len]
        time_emb: [batch, time_emb_dim]
        pos_embedding: [batch, 3, channels] - Subject, Verb, Object embeddings
        """
        residual = x
        
        # First conv block
        x = self.norm1(x)
        x = self.act(x)
        x = self.conv1(x)
        
        # Add time embedding
        time_emb = self.time_mlp(time_emb)
        x = x + time_emb[:, :, None]  # Broadcast across sequence
        
        # Second conv block
        x = self.norm2(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.conv2(x)
        
        # Add residual
        x = x + residual

        # Project POS to match current channel dimension
        pos_projected = self.pos_proj(pos_embedding)  # [batch, 3, channels]
        
        # Cross-attention to POS (work in [batch, seq_len, channels] format)
        x_transposed = x.transpose(1, 2)  # [batch, seq_len, channels]
        x_transposed = self.norm3(x_transposed)

        # print("x_transposed shape:", x_transposed.shape)
        # print("pos_embedding shape:", pos_embedding.shape)
        x_pos, _ = self.pos_cross_attn(
            query=x_transposed,      # Text attends to...
            key=pos_projected,       # POS embeddings
            value=pos_projected
        )
        
        x_transposed = x_transposed + x_pos
        x = x_transposed.transpose(1, 2)  # Back to [batch, channels, seq_len]
        
        return x

* Now let's write down block code (Using professor's code)

In [19]:
class DownBlock(nn.Module):
    """Downsampling block"""
    def __init__(self, in_channels, out_channels, dilation, time_emb_dim, pos_dim, num_heads=4):
        super().__init__()
        self.conv_block = DilatedConvBlock(in_channels, dilation, time_emb_dim, pos_dim, num_heads)
        self.downsample = nn.Conv1d(in_channels, out_channels, kernel_size=4, stride=2, padding=1)
        
    def forward(self, x, time_emb, pos_embedding):
        x = self.conv_block(x, time_emb, pos_embedding)
        skip = x
        x = self.downsample(x)
        return x, skip

* Now let's write code for up block (using professor's code)

In [20]:
class UpBlock(nn.Module):
    """Upsampling block with skip connections"""
    def __init__(self, in_channels, out_channels, dilation, time_emb_dim, pos_dim, num_heads=4):
        super().__init__()
        self.upsample = nn.ConvTranspose1d(in_channels, out_channels, kernel_size=4, stride=2, padding=1)
        # After concatenating with skip, we have out_channels * 2
        self.conv_block = DilatedConvBlock(out_channels * 2, dilation, time_emb_dim, pos_dim, num_heads)
        self.out_conv = nn.Conv1d(out_channels * 2, out_channels, kernel_size=1)
        
    def forward(self, x, skip, time_emb, pos_embedding):
        x = self.upsample(x)
        
        # Handle size mismatch between upsampled x and skip connection
        # This can happen due to odd-sized sequences and stride=2 operations
        if x.shape[2] != skip.shape[2]:
            # Crop or pad to match skip connection size
            if x.shape[2] < skip.shape[2]:
                # Pad x to match skip
                diff = skip.shape[2] - x.shape[2]
                x = F.pad(x, (0, diff))
            else:
                # Crop x to match skip
                x = x[:, :, :skip.shape[2]]
        
        # Concatenate skip connection
        x = torch.cat([x, skip], dim=1)
        x = self.conv_block(x, time_emb, pos_embedding)
        x = self.out_conv(x)
        return x

* Now we have all the utility classes for creating our Unet so let's create TextDiffusionUnet (Using professor's code)

In [21]:
class TextDiffusionUNet(nn.Module):
    """
    1D U-Net for text diffusion with POS conditioning
    """
    def __init__(
        self,
        hidden_dim=768,          # BERT hidden dimension
        time_emb_dim=256,        # Time embedding dimension
        channels=[768, 512, 256, 128],  # Channel progression through U-Net
        dilations=[1, 2, 4, 8],  # Dilation rates for each level
        num_heads=8,             # Number of attention heads
        dropout=0.1
    ):
        super().__init__()
        
        assert len(channels) == len(dilations), "channels and dilations must have same length"
        
        self.hidden_dim = hidden_dim
        self.time_emb_dim = time_emb_dim
        
        # Time embedding
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(time_emb_dim),
            nn.Linear(time_emb_dim, time_emb_dim * 4),
            nn.SiLU(),
            nn.Linear(time_emb_dim * 4, time_emb_dim)
        )
        
        # Initial projection
        self.input_proj = nn.Conv1d(hidden_dim, channels[0], kernel_size=1)
        
        # Encoder (downsampling path)
        self.down_blocks = nn.ModuleList()
        for i in range(len(channels) - 1):
            self.down_blocks.append(
                DownBlock(
                    channels[i],
                    channels[i + 1],
                    dilations[i],
                    time_emb_dim,
                    hidden_dim,  # POS dimension
                    num_heads
                )
            )
        
        # Bottleneck
        self.bottleneck = DilatedConvBlock(
            channels[-1],
            dilations[-1] * 2,  # Larger dilation at bottleneck
            time_emb_dim,
            hidden_dim,  # POS dimension
            num_heads,
            dropout
        )
        
        # Decoder (upsampling path)
        self.up_blocks = nn.ModuleList()
        for i in range(len(channels) - 1, 0, -1):
            self.up_blocks.append(
                UpBlock(
                    channels[i],
                    channels[i - 1],
                    dilations[i - 1],
                    time_emb_dim,
                    hidden_dim,  # POS dimension
                    num_heads
                )
            )
        
        # Final output projection
        self.output_proj = nn.Sequential(
            nn.GroupNorm(8, channels[0]),
            nn.SiLU(),
            nn.Conv1d(channels[0], hidden_dim, kernel_size=1)
        )
        
    def forward(self, x, timesteps, pos_embedding):
        """
        Args:
            x: [batch_size, seq_len, hidden_dim] - Noisy text embeddings
            timesteps: [batch_size] - Diffusion timesteps
            pos_embedding: [batch_size, 3, hidden_dim] - Subject, Verb, Object embeddings
            
        Returns:
            [batch_size, seq_len, hidden_dim] - Predicted noise
        """
        # Get time embeddings
        time_emb = self.time_mlp(timesteps)
        
        # Convert to [batch, channels, seq_len] for conv operations
        x = x.transpose(1, 2)  # [batch, hidden_dim, seq_len]
        
        # Initial projection
        x = self.input_proj(x)  # [batch, channels[0], seq_len]
        
        # Encoder with skip connections
        skips = []
        for down_block in self.down_blocks:
            x, skip = down_block(x, time_emb, pos_embedding)
            skips.append(skip)
            # print("Encoder: appending to skips block:", skip.shape)
        
        # Bottleneck
        x = self.bottleneck(x, time_emb, pos_embedding)
        
        # Decoder with skip connections
        for up_block in self.up_blocks:
            skip = skips.pop()
            # print("Decoder: popping from skips block:", skip.shape)
            x = up_block(x, skip, time_emb, pos_embedding)
        
        # Final projection
        x = self.output_proj(x)
        
        # Convert back to [batch, seq_len, hidden_dim]
        x = x.transpose(1, 2)
        
        return x

* Now let's write a function to create noise schedule

In [22]:
def create_noise_schedule(timesteps, beta1, beta2, device):
    """Function to create the noise schedule for diffusion"""
    b_t = (beta2 - beta1) * torch.linspace(0, 1, timesteps + 1, device=device) + beta1
    a_t = 1 - b_t
    ab_t = torch.cumsum(a_t.log(), dim=0).exp()
    ab_t[0] = 1
    return b_t, a_t, ab_t

* Let's now create a noise schedule with 1000 time steps

In [23]:
timesteps = 1000
beta1 = 2e-4
beta2 = 0.02

# Creating noise schedule
b_t, a_t, ab_t = create_noise_schedule(timesteps, beta1, beta2, device)

print(f"Noise schedule created:")
print(f"  ab_t[0] = {ab_t[0]:.4f} (start: ~100% signal)")
print(f"  ab_t[{timesteps//2}] = {ab_t[timesteps//2]:.4f} (middle)")
print(f"  ab_t[{timesteps}] = {ab_t[timesteps]:.4f} (end: ~0% signal)")

Noise schedule created:
  ab_t[0] = 1.0000 (start: ~100% signal)
  ab_t[500] = 0.0751 (middle)
  ab_t[1000] = 0.0000 (end: ~0% signal)


* Now let's write a function that adds noise to clean embeddings

In [24]:
def perturb_input(x, t, noise, ab_t):
    """Function for performing forward diffusion: Add noise to clean embeddings"""
    ab_t_vals = ab_t[t][:, None, None]  # [batch, 1, 1]
    return torch.sqrt(ab_t_vals) * x + torch.sqrt(1 - ab_t_vals) * noise

* Now let's write a function to remove predicted noise and add a small noise back

In [25]:
def denoise_add_noise(x, t, pred_noise, a_t, b_t, ab_t, z=None):
    """Function to perform reverse diffusion: Remove predicted noise (and add small noise back)"""
    if z is None:
        z = torch.randn_like(x)
    alpha_t = a_t[t]
    beta_t = b_t[t]
    alpha_bar_t = ab_t[t]
    noise_coeff = (1 - alpha_t) / torch.sqrt(1 - alpha_bar_t)
    mean = (x - noise_coeff * pred_noise) / torch.sqrt(alpha_t)
    noise = torch.sqrt(beta_t) * z
    return mean + noise

* Now let's create instance of a model

In [26]:
hidden_dim = 768
time_emb_dim = 256

model = TextDiffusionUNet(hidden_dim=hidden_dim, time_emb_dim=time_emb_dim, channels=[768, 512, 256, 128], dilations=[1, 2, 4, 8], num_heads=8, dropout=0.1).to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model parameters: {num_params:,}")

Model parameters: 63,567,360


* Let's write a function to train diffusion model

In [27]:
def train_diffusion_model(model, dataloader, optimizer, timesteps, a_t, b_t, ab_t, n_epoch, device, save_dir='./weights/', context_mask_prob=0.1, emb_mean=0.0, emb_std=1.0):
    """Training function for diffusion model"""
    model.train()
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    # Saving normalization stats
    torch.save({'emb_mean': emb_mean, 'emb_std': emb_std}, os.path.join(save_dir, 'norm_stats.pth'))
    print("\n" + "=" * 60)
    print("TRAINING START (FIXED)")
    print(f"Normalization: mean={emb_mean:.4f}, std={emb_std:.4f}")
    print("=" * 60)
    best_loss = float('inf')
    for epoch in range(n_epoch):
        model.train()
        # Learning rate with warmup and decay
        if epoch < 5:
            # Warmup
            lr = lrate * (epoch + 1) / 5
        else:
            # Cosine decay
            lr = lrate * 0.5 * (1 + math.cos(math.pi * (epoch - 5) / (n_epoch - 5)))
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        epoch_loss = 0
        epoch_mse = 0
        epoch_cos = 0
        num_batches = 0
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{n_epoch}")
        for batch in pbar:
            optimizer.zero_grad()
            x = batch['embeddings'].to(device)
            pos_emb = batch['pos_embeddings'].to(device)
            batch_size_actual = x.shape[0]
            # Context masking (classifier-free guidance)
            context_mask = torch.bernoulli(torch.zeros(batch_size_actual, device=device) + (1 - context_mask_prob))[:, None, None]
            pos_emb_masked = pos_emb * context_mask
            # Sampling noise
            noise = torch.randn_like(x)
            # Sampling timesteps
            t = torch.randint(1, timesteps + 1, (batch_size_actual,), device=device)
            # Forward diffusion
            x_noisy = perturb_input(x, t, noise, ab_t)
            # Predict noise
            t_normalized = t.float() / timesteps
            pred_noise = model(x_noisy, t_normalized, pos_emb_masked)
            # MSE loss
            mse_loss = F.mse_loss(pred_noise, noise)
            # Cosine loss
            pred_flat = pred_noise.reshape(-1, pred_noise.shape[-1])
            noise_flat = noise.reshape(-1, noise.shape[-1])
            cos_sim = F.cosine_similarity(pred_flat, noise_flat, dim=-1)
            cos_loss = (1 - cos_sim).mean()
            # Combined loss
            loss = mse_loss + 0.1 * cos_loss
            loss.backward()
            # Gradient clipping
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            epoch_loss += loss.item()
            epoch_mse += mse_loss.item()
            epoch_cos += cos_loss.item()
            num_batches += 1
            pbar.set_postfix({'loss': f'{loss.item():.4f}', 'mse': f'{mse_loss.item():.4f}', 'cos': f'{cos_loss.item():.4f}'})
        # Epoch statistics
        avg_loss = epoch_loss / num_batches
        avg_mse = epoch_mse / num_batches
        avg_cos = epoch_cos / num_batches
        print(f"Epoch {epoch+1}: Loss={avg_loss:.4f}, MSE={avg_mse:.4f}, "
              f"CosLoss={avg_cos:.4f}, LR={lr:.6f}")
        # Saving best model
        if avg_loss < best_loss:
            best_loss = avg_loss
            save_path = os.path.join(save_dir, "text_diffusion_best.pth")
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
                'emb_mean': emb_mean,
                'emb_std': emb_std,
            }, save_path)
            print(f"  New best model saved!")
    print("\n" + "=" * 60)
    print(f"TRAINING COMPLETE! Best loss: {best_loss:.4f}")
    print("=" * 60)

* Now that we have writrten training function let's train our model

In [28]:
n_epoch = 300
save_dir = './weights/'
lrate = 1e-4

In [29]:
# Creating optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lrate)
# Training diffusion model
train_diffusion_model(model=model, dataloader=dataloader, optimizer=optimizer, timesteps=timesteps, a_t=a_t, b_t=b_t, ab_t=ab_t, n_epoch=n_epoch, device=device, save_dir=save_dir, context_mask_prob=0.1)


TRAINING START (FIXED)
Normalization: mean=0.0000, std=1.0000


Epoch 1/300: 100%|██████████| 102/102 [00:03<00:00, 28.40it/s, loss=1.0789, mse=0.9892, cos=0.8977]


Epoch 1: Loss=1.1031, MSE=1.0081, CosLoss=0.9493, LR=0.000020
  New best model saved!


Epoch 2/300: 100%|██████████| 102/102 [00:03<00:00, 31.16it/s, loss=0.9187, mse=0.8575, cos=0.6116]


Epoch 2: Loss=1.0002, MSE=0.9261, CosLoss=0.7406, LR=0.000040
  New best model saved!


...

Epoch 298: Loss=0.1356, MSE=0.1279, CosLoss=0.0771, LR=0.000000


Epoch 299/300: 100%|██████████| 102/102 [00:03<00:00, 32.30it/s, loss=0.1148, mse=0.1077, cos=0.0702]


Epoch 299: Loss=0.1368, MSE=0.1290, CosLoss=0.0783, LR=0.000000


Epoch 300/300: 100%|██████████| 102/102 [00:03<00:00, 32.12it/s, loss=0.2120, mse=0.1995, cos=0.1244]

Epoch 300: Loss=0.1437, MSE=0.1354, CosLoss=0.0829, LR=0.000000

TRAINING COMPLETE! Best loss: 0.1294


* Now let's write decoder class for decoding embeddings back to tokens (Using professor's code)

In [30]:
class LatentToTextDecoder:
    """Converts denoised BERT latent vectors back to text tokens"""
    
    def __init__(self, model_name='bert-base-uncased', device='cuda'):
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.bert = AutoModel.from_pretrained(model_name).to(device)
        self.bert.eval()
        
        # Get BERT's token embedding matrix
        self.token_embeddings = self.bert.embeddings.word_embeddings.weight.data
        self.vocab_size, self.hidden_dim = self.token_embeddings.shape
        
        print(f"Decoder initialized: vocab_size={self.vocab_size}, hidden_dim={self.hidden_dim}")
    
    @torch.no_grad()
    def decode_nearest_neighbor(self, denoised_latents, skip_special_tokens=True):
        """
        Decode using nearest neighbor search
        
        Args:
            denoised_latents: [batch, seq_len, hidden_dim]
            skip_special_tokens: Skip [CLS], [SEP], [PAD] in output
            
        Returns:
            List of decoded text strings
        """
        denoised_latents = denoised_latents.to(self.device)
        
        # Normalize for cosine similarity
        denoised_norm = F.normalize(denoised_latents, dim=-1)
        token_emb_norm = F.normalize(self.token_embeddings, dim=-1)
        
        # Compute similarities
        similarities = torch.matmul(denoised_norm, token_emb_norm.T)
        
        # Get nearest token IDs
        token_ids = similarities.argmax(dim=-1)
        
        # Decode to text
        texts = []
        for i in range(token_ids.shape[0]):
            tokens = token_ids[i].cpu().tolist()
            text = self.tokenizer.decode(tokens, skip_special_tokens=skip_special_tokens)
            texts.append(text)
        
        return texts
    
    @torch.no_grad()
    def decode_topk_sampling(self, denoised_latents, k=5, temperature=1.0):
        """
        Decode with top-k sampling for diversity
        """
        denoised_latents = denoised_latents.to(self.device)
        batch_size, seq_len, _ = denoised_latents.shape
        
        denoised_norm = F.normalize(denoised_latents, dim=-1)
        token_emb_norm = F.normalize(self.token_embeddings, dim=-1)
        
        similarities = torch.matmul(denoised_norm, token_emb_norm.T)
        logits = similarities / temperature
        
        topk_logits, topk_indices = torch.topk(logits, k, dim=-1)
        topk_probs = F.softmax(topk_logits, dim=-1)
        
        token_ids = []
        for i in range(batch_size):
            batch_tokens = []
            for j in range(seq_len):
                idx = torch.multinomial(topk_probs[i, j], 1)
                token_id = topk_indices[i, j, idx]
                batch_tokens.append(token_id.item())
            token_ids.append(batch_tokens)
        
        texts = []
        for tokens in token_ids:
            text = self.tokenizer.decode(tokens, skip_special_tokens=True)
            texts.append(text)
        
        return texts

* Now let's write a function to get POS embedding while generating output using our model

In [31]:
@torch.no_grad()
def get_pos_embedding_for_generation(words, tokenizer, bert_model, n_pos_words, device):
    """Function to get CONTEXTUALIZED POS embeddings for generation"""
    words = list(words)
    # Pad if needed
    while len(words) < n_pos_words:
        words.append(tokenizer.pad_token if tokenizer.pad_token else '[PAD]')
    words = words[:n_pos_words]
    # Join and run BERT on POS words only
    pos_text = ' '.join(words)
    encoded = tokenizer(pos_text, return_tensors='pt', padding='max_length', truncation=True, max_length=n_pos_words + 2).to(device)
    outputs = bert_model(**encoded)
    # Skipping [CLS], take n_pos_words embeddings
    pos_emb = outputs.last_hidden_state.squeeze(0)[1:n_pos_words + 1, :]
    return pos_emb

* Now let's write a sampling function which also denormalizes embedding back

In [32]:
@torch.no_grad()
def sample_ddpm(model, n_sample, seq_len, pos_embedding, timesteps, a_t, b_t, ab_t, device, emb_mean, emb_std):
    """Function to perform sampling using ddpm and also denormalize embeddings back after denoising"""
    model.eval()
    # Starting from pure noise
    samples = torch.randn(n_sample, seq_len, model.hidden_dim).to(device)
    print(f"  Initial noise: mean={samples.mean():.4f}, std={samples.std():.4f}")
    for i in range(timesteps, 0, -1):
        if i % 100 == 0:
            print(f"  Step {i:3d}/{timesteps}, "
                  f"mean={samples.mean():.4f}, std={samples.std():.4f}")    
        t_normalized = torch.tensor([i / timesteps] * n_sample, device=device)
        z = torch.randn_like(samples) if i > 1 else 0
        pred_noise = model(samples, t_normalized, pos_embedding)
        samples = denoise_add_noise(samples, i, pred_noise, a_t, b_t, ab_t, z)
    print(f"  Final (normalized): mean={samples.mean():.4f}, std={samples.std():.4f}")
    # Denormalizing back to BERT scale
    samples = samples * emb_std + emb_mean
    print(f"  After denormalization: mean={samples.mean():.4f}, std={samples.std():.4f}")
    return samples

* For decoding embedding back to text we are going to train custom decoder
* So let's write a custom decoder class

In [33]:
class ContextualEmbeddingDecoder(nn.Module):
    """Neural network that learns to map BERT contextual embeddings back to token IDs."""
    def __init__(self, hidden_dim=768, vocab_size=30522, intermediate_dim=1024, n_layers=3, dropout=0.1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        # Input normalization
        self.input_norm = nn.LayerNorm(hidden_dim)
        # Build transformation layers
        layers = []
        # First layer: hidden_dim -> intermediate_dim
        layers.append(nn.Linear(hidden_dim, intermediate_dim))
        layers.append(nn.LayerNorm(intermediate_dim))
        layers.append(nn.GELU())
        layers.append(nn.Dropout(dropout))
        # Middle layers: intermediate_dim -> intermediate_dim
        for _ in range(n_layers - 2):
            layers.append(nn.Linear(intermediate_dim, intermediate_dim))
            layers.append(nn.LayerNorm(intermediate_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))
        # Final hidden layer: intermediate_dim -> hidden_dim
        layers.append(nn.Linear(intermediate_dim, hidden_dim))
        layers.append(nn.LayerNorm(hidden_dim))
        layers.append(nn.GELU())
        self.transform = nn.Sequential(*layers)
        # Outputing projection to vocabulary
        self.output_proj = nn.Linear(hidden_dim, vocab_size)
        # Residual connection weight
        self.residual_weight = nn.Parameter(torch.tensor(0.1))
        
    def forward(self, x):
        # Normalizing input
        x_norm = self.input_norm(x)
        # Transform
        transformed = self.transform(x_norm)
        # Residual connection
        combined = transformed + self.residual_weight * x_norm
        # Projecting to vocabulary
        logits = self.output_proj(combined)
        return logits

* Now that we have a decoder network which maps back contextualized bert embedding back to token Id let's also write a class for training it

In [34]:
class DecoderTrainer:
    """Handles training of the ContextualEmbeddingDecoder."""
    def __init__(self, model_name='bert-base-uncased', device='cuda'):
        self.device = device
        self.model_name = model_name
        # Loading BERT
        print(f"Loading BERT model: {model_name}")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.bert = AutoModel.from_pretrained(model_name).to(device)
        self.bert.eval()
        # Freezing BERT
        for param in self.bert.parameters():
            param.requires_grad = False
        self.hidden_dim = self.bert.config.hidden_size
        self.vocab_size = self.bert.config.vocab_size
        print(f"Hidden dim: {self.hidden_dim}, Vocab size: {self.vocab_size}")
    
    def prepare_training_data(self, texts, max_length=256):
        """Function to prepare training data: get BERT embeddings and corresponding token IDs."""
        print(f"Preparing training data for {len(texts)} texts.")
        all_embeddings = []
        all_token_ids = []
        all_attention_masks = []
        self.bert.eval()
        with torch.no_grad():
            for text in tqdm(texts):
                # Tokenize
                encoded = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length).to(self.device)
                # Geting BERT embeddings
                outputs = self.bert(**encoded)
                embeddings = outputs.last_hidden_state.squeeze(0).cpu()
                # Store
                all_embeddings.append(embeddings)
                all_token_ids.append(encoded['input_ids'].squeeze(0).cpu())
                all_attention_masks.append(encoded['attention_mask'].squeeze(0).cpu())
        print(f"  Done! Prepared {len(all_embeddings)} samples.")
        return all_embeddings, all_token_ids, all_attention_masks
    
    def train(self, texts, decoder=None, n_epochs=20, batch_size=32, lr=1e-4, noise_schedule='progressive', max_noise=0.5, max_length=256, save_path=None):
        """Function to train the decoder network."""
        # Preparing data
        all_embeddings, all_token_ids, all_attention_masks = self.prepare_training_data(texts, max_length)
        # Creating decoder if not provided
        if decoder is None:
            decoder = ContextualEmbeddingDecoder(hidden_dim=self.hidden_dim, vocab_size=self.vocab_size, intermediate_dim=1024, n_layers=3, dropout=0.1).to(self.device)
        # Countimg parameters
        n_params = sum(p.numel() for p in decoder.parameters() if p.requires_grad)
        print(f"\nDecoder parameters: {n_params:,}")
        # Optimizer and loss
        optimizer = torch.optim.AdamW(decoder.parameters(), lr=lr, weight_decay=0.01)
        criterion = nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
        # Learning rate scheduler
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)
        print(f"\n{'='*60}")
        print("TRAINING DECODER")
        print(f"{'='*60}")
        print(f"Epochs: {n_epochs}")
        print(f"Batch size: {batch_size}")
        print(f"Learning rate: {lr}")
        print(f"Noise schedule: {noise_schedule}")
        print(f"Max noise: {max_noise}")
        # Training loop
        n_samples = len(all_embeddings)
        for epoch in range(n_epochs):
            decoder.train()
            # Determine noise level for this epoch
            if noise_schedule == 'progressive':
                # Starting with low noise then gradually increase noise
                epoch_noise = max_noise * (epoch + 1) / n_epochs
            else:
                epoch_noise = max_noise
            # Shuffling data
            indices = torch.randperm(n_samples)
            total_loss = 0
            total_correct = 0
            total_tokens = 0
            n_batches = 0
            for batch_start in range(0, n_samples, batch_size):
                batch_indices = indices[batch_start:batch_start + batch_size] 
                # Gathering batch data
                batch_embeddings = torch.stack([all_embeddings[i] for i in batch_indices]).to(self.device)
                batch_token_ids = torch.stack([all_token_ids[i] for i in batch_indices]).to(self.device)
                batch_attention_mask = torch.stack([all_attention_masks[i] for i in batch_indices]).to(self.device)
                # Adding noise
                noise_level = torch.rand(1).item() * epoch_noise
                noise = torch.randn_like(batch_embeddings) * noise_level
                noisy_embeddings = batch_embeddings + noise
                # Forward pass
                logits = decoder(noisy_embeddings)
                # Computing loss
                loss = criterion(logits.view(-1, self.vocab_size), batch_token_ids.view(-1))
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(decoder.parameters(), 1.0)
                optimizer.step()
                # Tracking metrics
                total_loss += loss.item()
                # Accuracy (on non-padded tokens)
                predictions = logits.argmax(dim=-1)
                mask = batch_attention_mask.bool()
                correct = ((predictions == batch_token_ids) & mask).sum().item()
                total_correct += correct
                total_tokens += mask.sum().item()
                n_batches += 1
            # Updating learning rate
            scheduler.step()
            # Epoch statistics
            avg_loss = total_loss / n_batches
            accuracy = total_correct / total_tokens * 100
            print(f"Epoch {epoch + 1:3d}/{n_epochs} | "
                  f"Loss: {avg_loss:.4f} | "
                  f"Acc: {accuracy:.2f}% | "
                  f"Noise: {epoch_noise:.3f} | "
                  f"LR: {scheduler.get_last_lr()[0]:.6f}")
        # Save decoder
        if save_path:
            torch.save({
                'model_state_dict': decoder.state_dict(),
                'config': {
                    'hidden_dim': self.hidden_dim,
                    'vocab_size': self.vocab_size,
                    'model_name': self.model_name
                }
            }, save_path)
            print(f"\nDecoder saved to: {save_path}")
        print(f"\n{'='*60}")
        print("DECODER TRAINING COMPLETE")
        print(f"{'='*60}")
        return decoder

* Now let's train our decoder model

In [35]:
# Preparing texts for decoder training
decoder_texts = [sample['text'] for sample in all_data_samples]
trainer = DecoderTrainer(model_name=model_name, device=device)
decoder = trainer.train(texts=decoder_texts, n_epochs=20, batch_size=32, lr=1e-4, noise_schedule='progressive', max_noise=0.5, max_length=max_length, save_path=os.path.join(save_dir, 'decoder.pth'))

Loading BERT model: bert-base-uncased
Hidden dim: 768, Vocab size: 30522
Preparing training data for 3274 texts.


100%|██████████| 3274/3274 [00:12<00:00, 255.11it/s]


  Done! Prepared 3274 samples.

Decoder parameters: 26,102,843

TRAINING DECODER
Epochs: 20
Batch size: 32
Learning rate: 0.0001
Noise schedule: progressive
Max noise: 0.5
Epoch   1/20 | Loss: 6.0264 | Acc: 35.42% | Noise: 0.025 | LR: 0.000099
Epoch   2/20 | Loss: 2.9795 | Acc: 61.56% | Noise: 0.050 | LR: 0.000098
Epoch   3/20 | Loss: 2.0968 | Acc: 72.14% | Noise: 0.075 | LR: 0.000095
Epoch   4/20 | Loss: 1.5827 | Acc: 79.32% | Noise: 0.100 | LR: 0.000090
Epoch   5/20 | Loss: 1.2131 | Acc: 84.84% | Noise: 0.125 | LR: 0.000085
Epoch   6/20 | Loss: 0.9526 | Acc: 89.43% | Noise: 0.150 | LR: 0.000079
Epoch   7/20 | Loss: 0.7542 | Acc: 93.24% | Noise: 0.175 | LR: 0.000073
Epoch   8/20 | Loss: 0.6034 | Acc: 95.71% | Noise: 0.200 | LR: 0.000065
Epoch   9/20 | Loss: 0.4949 | Acc: 97.31% | Noise: 0.225 | LR: 0.000058
Epoch  10/20 | Loss: 0.4132 | Acc: 98.24% | Noise: 0.250 | LR: 0.000050
Epoch  11/20 | Loss: 0.3654 | Acc: 98.78% | Noise: 0.275 | LR: 0.000042
Epoch  12/20 | Loss: 0.3200 | Acc: 9

* Now let's write a text generator class to generate text using model above and decoder

In [36]:
class TextGenerator:
    """Class which contains functions to generate text using diffusion model and decoder"""
    def __init__(self, diffusion_model, decoder, tokenizer, bert_model, emb_mean, emb_std, device='cuda'):
        self.diffusion_model = diffusion_model
        self.decoder = decoder
        self.tokenizer = tokenizer
        self.bert = bert_model
        self.emb_mean = emb_mean
        self.emb_std = emb_std
        self.device = device
        self.diffusion_model.eval()
        self.decoder.eval()
        self.bert.eval()
        print(f"Generator initialized with normalization: "
              f"mean={emb_mean:.4f}, std={emb_std:.4f}")
        
    @torch.no_grad()
    def generate(self, pos_word_lists, seq_len=64, timesteps=1000, a_t=None, b_t=None, ab_t=None, n_pos_words=3, temperature=0.8, top_k=50, top_p=0.9):
        """Function to generate text"""
        n_samples = len(pos_word_lists)
        print(f"\nGenerating {n_samples} samples...")
        # Geting POS embeddings
        print("Encoding POS words...")
        pos_embeddings = []
        for words in pos_word_lists:
            pos_emb = get_pos_embedding_for_generation(words, self.tokenizer, self.bert, n_pos_words, self.device)
            # Normalizing POS embeddings to match training
            pos_emb = (pos_emb - self.emb_mean) / self.emb_std
            pos_embeddings.append(pos_emb)
        pos_embedding = torch.stack(pos_embeddings).to(self.device)
        print(f"  POS embedding shape: {pos_embedding.shape}")
        print(f"  POS embedding stats (normalized): "
              f"mean={pos_embedding.mean():.4f}, std={pos_embedding.std():.4f}")
        # Running diffusion with fixed sampling
        print(f"Running diffusion ({timesteps} steps)...")
        samples = sample_ddpm(model=self.diffusion_model, n_sample=n_samples, seq_len=seq_len, pos_embedding=pos_embedding, timesteps=timesteps, a_t=a_t, b_t=b_t, ab_t=ab_t, device=self.device, emb_mean=self.emb_mean, emb_std=self.emb_std)   
        # Decoding
        print("Decoding to text...")
        logits = self.decoder(samples)
        # Applying temperature and top-k/top-p sampling
        logits = logits / temperature
        if top_k > 0:
            topk_vals, _ = torch.topk(logits, min(top_k, logits.size(-1)), dim=-1)
            threshold = topk_vals[..., -1, None]
            logits = torch.where(logits < threshold, torch.full_like(logits, float('-inf')), logits)
        if top_p < 1.0:
            sorted_logits, sorted_indices = torch.sort(logits, descending=True, dim=-1)
            cumsum = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
            mask = cumsum > top_p
            mask[..., 1:] = mask[..., :-1].clone()
            mask[..., 0] = False
            indices_to_remove = mask.scatter(-1, sorted_indices, mask)
            logits = logits.masked_fill(indices_to_remove, float('-inf'))
        probs = F.softmax(logits, dim=-1)
        token_ids = torch.multinomial(probs.view(-1, probs.size(-1)), 1).view(n_samples, seq_len)
        # Converting to text
        texts = []
        for i in range(n_samples):
            text = self.tokenizer.decode(token_ids[i].cpu().tolist(), skip_special_tokens=True)
            texts.append(text)
        return texts, samples

* Now let's test this Text geneartaor class

In [37]:
# Reload BERT for generation
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name).to(device)
bert_model.eval()

generator = TextGenerator(diffusion_model=model, decoder=decoder, tokenizer=tokenizer, bert_model=bert_model, device=device, emb_mean=dataset.get_normalization_stats()[0], emb_std=dataset.get_normalization_stats()[1])

test_pos = [
    ["Shall", "I", "compare"],
    ["When", "I", "do"],
    ["Love", "is", "not"],
]

texts, embeddings = generator.generate(pos_word_lists=test_pos, seq_len=max_length, timesteps=timesteps, a_t=a_t, b_t=b_t, ab_t=ab_t, n_pos_words=n_pos_words, temperature=0.8, top_k=50, top_p=0.9)

for i, (pos, text) in enumerate(zip(test_pos, texts)):
    print(f"\nSample {i + 1}")
    print(f"POS: {pos}")
    print(f"Generated: {text[:500]}")
    print("-" * 40)

Generator initialized with normalization: mean=-0.0089, std=0.4152

Generating 3 samples...
Encoding POS words...
  POS embedding shape: torch.Size([3, 3, 768])
  POS embedding stats (normalized): mean=-0.0057, std=1.2395
Running diffusion (1000 steps)...
  Initial noise: mean=-0.0043, std=0.9970
  Step 1000/1000, mean=-0.0043, std=0.9970
  Step 900/1000, mean=-0.0010, std=1.5945
  Step 800/1000, mean=0.0319, std=7.0510
  Step 700/1000, mean=0.1310, std=19.9197
  Step 600/1000, mean=0.3090, std=42.9433
  Step 500/1000, mean=0.5947, std=78.7806
  Step 400/1000, mean=0.9812, std=127.5995
  Step 300/1000, mean=1.4358, std=185.0478
  Step 200/1000, mean=1.8839, std=241.7176
  Step 100/1000, mean=2.2279, std=285.1319
  Final (normalized): mean=2.3726, std=303.7939
  After denormalization: mean=0.9761, std=126.1201
Decoding to text...

Sample 1
POS: ['Shall', 'I', 'compare']
Generated: your speak, let it checkzing my be much aug where shade longer who, almost ; pencil to dark show might, wan

* Let's also write a function that will generate poem from saved model and decoder

In [38]:
def generate_from_saved_models(diffusion_path, decoder_path, pos_word_lists):
    """Function to generate using saved models with normalization."""
    print("\n" + "=" * 60)
    print("GENERATING FROM SAVED MODELS")
    print("=" * 60)
    # Loading normalization stats
    norm_stats_path = os.path.join(save_dir, 'norm_stats.pth')
    if os.path.exists(norm_stats_path):
        norm_stats = torch.load(norm_stats_path)
        emb_mean = norm_stats['emb_mean']
        emb_std = norm_stats['emb_std']
    else:
        # Trying to get from checkpoint
        checkpoint = torch.load(diffusion_path, map_location=device)
        emb_mean = checkpoint.get('emb_mean', 0.0)
        emb_std = checkpoint.get('emb_std', 0.4)
    print(f"Normalization stats: mean={emb_mean:.4f}, std={emb_std:.4f}")
    # Load models
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    bert_model = AutoModel.from_pretrained(model_name).to(device)
    bert_model.eval()
    diffusion_model = TextDiffusionUNet(hidden_dim=hidden_dim, time_emb_dim=time_emb_dim, channels=[768, 512, 256, 128], dilations=[1, 2, 4, 8], num_heads=8, dropout=0.1).to(device)
    checkpoint = torch.load(diffusion_path, map_location=device)
    diffusion_model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Loaded diffusion model from epoch {checkpoint['epoch']}")
    decoder_checkpoint = torch.load(decoder_path, map_location=device)
    decoder = ContextualEmbeddingDecoder(hidden_dim=decoder_checkpoint['config']['hidden_dim'], vocab_size=decoder_checkpoint['config']['vocab_size'], intermediate_dim=1024, n_layers=3, dropout=0.1).to(device)
    decoder.load_state_dict(decoder_checkpoint['model_state_dict'])
    # Creating generator
    generator = TextGenerator(diffusion_model=diffusion_model, decoder=decoder, tokenizer=tokenizer, bert_model=bert_model, emb_mean=emb_mean, emb_std=emb_std, device=device)
    
    # Generate
    texts, embeddings = generator.generate(pos_word_lists=pos_word_lists, seq_len=max_length, timesteps=timesteps, a_t=a_t, b_t=b_t, ab_t=ab_t, n_pos_words=n_pos_words, temperature=0.8, top_k=50, top_p=0.9)
    
    # Printing results
    print("\n" + "=" * 60)
    print("GENERATED TEXTS")
    print("=" * 60)
    
    for i, (pos, text) in enumerate(zip(pos_word_lists, texts)):
        print(f"\n{'='*50}")
        print(f"Sample {i + 1}: {pos}")
        print(f"{'='*50}")
        print(text)
    
    return texts, embeddings

* Now let's also test this function

In [39]:
pos_word_lists = [
        ["Shall", "I", "compare"],
        ["When", "I", "do"],
        ["Love", "is", "not"],
        ["My", "love", "is"],
        ["From", "fairest", "creatures"],
]

In [40]:
texts, embeddings = generate_from_saved_models(
    diffusion_path=os.path.join(save_dir, f'text_diffusion_best.pth'),
    decoder_path=os.path.join(save_dir, 'decoder.pth'),
    pos_word_lists=pos_word_lists
)


GENERATING FROM SAVED MODELS
Normalization stats: mean=0.0000, std=1.0000
Loaded diffusion model from epoch 293
Generator initialized with normalization: mean=0.0000, std=1.0000

Generating 5 samples...
Encoding POS words...
  POS embedding shape: torch.Size([5, 3, 768])
  POS embedding stats (normalized): mean=-0.0117, std=0.5249
Running diffusion (1000 steps)...
  Initial noise: mean=-0.0013, std=0.9978
  Step 1000/1000, mean=-0.0013, std=0.9978
  Step 900/1000, mean=0.0027, std=1.5957
  Step 800/1000, mean=0.0389, std=7.0248
  Step 700/1000, mean=0.1382, std=19.6239
  Step 600/1000, mean=0.3261, std=42.1497
  Step 500/1000, mean=0.6143, std=77.2012
  Step 400/1000, mean=1.0098, std=124.9193
  Step 300/1000, mean=1.4756, std=181.0275
  Step 200/1000, mean=1.9342, std=236.3064
  Step 100/1000, mean=2.2839, std=278.5316
  Final (normalized): mean=2.4275, std=296.4676
  After denormalization: mean=2.4275, std=296.4676
Decoding to text...

GENERATED TEXTS

Sample 1: ['Shall', 'I', 'comp

* Now let's write a function to generate poem using model

In [41]:
def generate_poem_diffusion(diffusion_path, decoder_path, seed_text, num_samples=1, 
                            seq_len=20, temperature=0.8, top_k=50, top_p=0.9):
    """Function to generate poem using diffusion model"""
    # Load normalization stats
    norm_stats_path = os.path.join(save_dir, 'norm_stats.pth')
    if os.path.exists(norm_stats_path):
        norm_stats = torch.load(norm_stats_path)
        emb_mean = norm_stats['emb_mean']
        emb_std = norm_stats['emb_std']
    else:
        checkpoint = torch.load(diffusion_path, map_location=device)
        emb_mean = checkpoint.get('emb_mean', 0.0)
        emb_std = checkpoint.get('emb_std', 0.4)
    
    # Load models
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    bert_model = AutoModel.from_pretrained(model_name).to(device)
    bert_model.eval()
    
    # Load diffusion model
    diffusion_model = TextDiffusionUNet(
        hidden_dim=hidden_dim, 
        time_emb_dim=time_emb_dim, 
        channels=[768, 512, 256, 128], 
        dilations=[1, 2, 4, 8], 
        num_heads=8, 
        dropout=0.1
    ).to(device)
    
    checkpoint = torch.load(diffusion_path, map_location=device)
    diffusion_model.load_state_dict(checkpoint['model_state_dict'])
    diffusion_model.eval()
    
    # Load decoder
    decoder_checkpoint = torch.load(decoder_path, map_location=device)
    decoder = ContextualEmbeddingDecoder(
        hidden_dim=decoder_checkpoint['config']['hidden_dim'],
        vocab_size=decoder_checkpoint['config']['vocab_size'],
        intermediate_dim=1024,
        n_layers=3,
        dropout=0.1
    ).to(device)
    decoder.load_state_dict(decoder_checkpoint['model_state_dict'])
    decoder.eval()
    
    # Create generator
    generator = TextGenerator(
        diffusion_model=diffusion_model,
        decoder=decoder,
        tokenizer=tokenizer,
        bert_model=bert_model,
        emb_mean=emb_mean,
        emb_std=emb_std,
        device=device
    )
    
    # Convert seed text to word list
    seed_words = seed_text.strip().split()
    
    # Create pos_word_lists for multiple samples
    pos_word_lists = [seed_words for _ in range(num_samples)]
    
    # Generate
    texts, embeddings = generator.generate(
        pos_word_lists=pos_word_lists,
        seq_len=seq_len,
        timesteps=timesteps,
        a_t=a_t,
        b_t=b_t,
        ab_t=ab_t,
        n_pos_words=len(seed_words),
        temperature=temperature,
        top_k=top_k,
        top_p=top_p
    )
    
    return texts

In [42]:
def format_poem(text, words_per_line=8):
    """Function to format text into poem lines."""
    # Clean up the text
    words = text.replace(',', ' ,').replace('.', ' .').split()
    words = [w for w in words if w.strip()]
    
    lines = []
    for i in range(0, len(words), words_per_line):
        line = ' '.join(words[i:i + words_per_line])
        # Clean up spacing around punctuation
        line = line.replace(' ,', ',').replace(' .', '.')
        lines.append(line)
    
    return '\n'.join(lines)

In [43]:
# Test diffusion poem generation
poems = generate_poem_diffusion(
    diffusion_path=os.path.join(save_dir, f'text_diffusion_best.pth'),
    decoder_path=os.path.join(save_dir, 'decoder.pth'),
    seed_text="shall i compare thee",
    seq_len=100
)

for i, poem in enumerate(poems):
    print(f"\n--- Generated Poem ---")
    print(format_poem(poem))

Generator initialized with normalization: mean=0.0000, std=1.0000

Generating 1 samples...
Encoding POS words...
  POS embedding shape: torch.Size([1, 4, 768])
  POS embedding stats (normalized): mean=-0.0126, std=0.5501
Running diffusion (1000 steps)...
  Initial noise: mean=-0.0018, std=1.0007
  Step 1000/1000, mean=-0.0018, std=1.0007
  Step 900/1000, mean=0.0017, std=1.3833
  Step 800/1000, mean=0.0260, std=5.9822
  Step 700/1000, mean=0.0987, std=17.2368
  Step 600/1000, mean=0.2398, std=37.3679
  Step 500/1000, mean=0.4702, std=68.7003
  Step 400/1000, mean=0.7848, std=111.3670
  Step 300/1000, mean=1.1591, std=161.5587
  Step 200/1000, mean=1.5312, std=211.0282
  Step 100/1000, mean=1.8126, std=248.8259
  Final (normalized): mean=1.9257, std=264.8960
  After denormalization: mean=1.9257, std=264.8960
Decoding to text...

--- Generated Poem ---
beauty. that fashion to no to '
measure are, every, of as things
nothing merit, tomb the my an,
poets,sw my have full beauty be receives
